In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy
from decimal import *
import pandas as pd

In [2]:
import threading
import numpy as np
import datetime

In [3]:
import itertools

In [4]:
#ORIGINAL
# setting up a stake matrix for a user-defined number of nodes
# random number of channels per node between min and max params


def setupStake(
        # Number of nodes in the network. This number can be changed to test with small and large network size
        numNodes=4,
        # the minimum number of channels a node can open
        minChannelsPerNode=2,
        # the maximum number of channels a node can open 
        maxChannelsPerNode=10,
        minFundsPerNode=10,
        maxFundsPerNode=100,
        tokensPerTicket=0.1
    ):

    stake = [[0 for i in range(numNodes)] for j in range(numNodes)]
    for x in range(numNodes):
        # each node is given a random funding amount
        myFunds = numpy.random.rand() * (maxFundsPerNode - minFundsPerNode) + minFundsPerNode

        # get random number of channels per node
        myChannels = int(numpy.random.rand() * (maxChannelsPerNode - minChannelsPerNode + 1) + minChannelsPerNode)

        # This value represents the amount each node stakes in their channel
        # It is computed as the number of funds a node has divided by number of channels they open
        stakePerChannel = myFunds / myChannels
        stakePerChannel = int(stakePerChannel / tokensPerTicket) * tokensPerTicket

        # fund channels by writing into stake matrix
        for c in range(myChannels):
            # TODO: this does not prevent a node from opening a channel to the same counterparty multiple times
            counterparty = int(numpy.random.rand() * (numNodes - 1))

            # cannot open channel to self - keep diagonal of matrix at 0
            if counterparty >= x:
                counterparty = counterparty + 1
            stake[x][counterparty] = stakePerChannel
            stake = [[Decimal(i) for i in j] for j in stake]

    return stake

In [5]:
a = setupStake()
a

[[Decimal('0'),
  Decimal('7.60000000000000053290705182007513940334320068359375'),
  Decimal('7.60000000000000053290705182007513940334320068359375'),
  Decimal('7.60000000000000053290705182007513940334320068359375')],
 [Decimal('14.9000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('14.9000000000000003552713678800500929355621337890625'),
  Decimal('14.9000000000000003552713678800500929355621337890625')],
 [Decimal('4.60000000000000053290705182007513940334320068359375'),
  Decimal('4.60000000000000053290705182007513940334320068359375'),
  Decimal('0'),
  Decimal('4.60000000000000053290705182007513940334320068359375')],
 [Decimal('47.2000000000000028421709430404007434844970703125'),
  Decimal('0'),
  Decimal('47.2000000000000028421709430404007434844970703125'),
  Decimal('0')]]

In [6]:
def calcImportance(stake):

    # get total stake per node
    stakePerNode = [Decimal(sum(e)) for e in stake]

    totalStake = sum(stakePerNode)

    n = len(stake)
    weight = [0] * n
    importance = [0] * n
    for x in range(n):
        for y in range(n):
            # weight of a node is 0 if its stake is equal to 0
            #weight(channel) = sqrt(balance(channel) / stake(channel.source) * stake(channel.destination))
            weight[y] += 0 if stakePerNode[y]==0 else numpy.sqrt(stake[y][x]/stakePerNode[y] * stakePerNode[x])

    #print("Weighted downstream stake per node p(n) = ", weight)
    importanceList = numpy.array(weight) * numpy.array(stakePerNode)
    return importanceList;

In [7]:
calcImportance(a)

array([Decimal('264.8066920085565162471297935'),
       Decimal('469.8455680769878139753695303'),
       Decimal('168.7240065789793524983772417'),
       Decimal('566.6995982749350774123416642')], dtype=object)

In [8]:
#in the given form the importance score does not depend on the number of potential channels, see below - the scores are equal with 4x4 and 5x5 matrix if the 5th node doesn't have any channel 

In [9]:
#CHCKING if the potential number of channels (even if it's empty) influencing the importance score

# Number of nodes in the network. This number can be changed to test with small and large network size
numNodes=4
# the minimum number of channels a node can open
minChannelsPerNode=2
# the maximum number of channels a node can open 
maxChannelsPerNode=10
minFundsPerNode=10
maxFundsPerNode=100
tokensPerTicket=0.1

def setupStake():
    stake = [[0 for i in range(numNodes)] for j in range(numNodes)]
    for x in range(numNodes):
        # each node is given a random funding amount
        myFunds = numpy.random.rand() * (maxFundsPerNode - minFundsPerNode) + minFundsPerNode

        # get random number of channels per node
        myChannels = int(numpy.random.rand() * (maxChannelsPerNode - minChannelsPerNode + 1) + minChannelsPerNode)

        # This value represents the amount each node stakes in their channel
        # It is computed as the number of funds a node has divided by number of channels they open
        stakePerChannel = myFunds / myChannels
        stakePerChannel = int(stakePerChannel / tokensPerTicket) * tokensPerTicket

        # fund channels by writing into stake matrix
        for c in range(myChannels):
            # TODO: this does not prevent a node from opening a channel to the same counterparty multiple times
            counterparty = int(numpy.random.rand() * (numNodes - 1))

            # cannot open channel to self - keep diagonal of matrix at 0
            if counterparty >= x:
                counterparty = counterparty + 1
            stake[x][counterparty] = stakePerChannel
            stake = [[Decimal(i) for i in j] for j in stake]
    return stake
            
def setupStake_e():
    stake2 = [[0 for i in range(numNodes+1)] for j in range(numNodes+1)]
    stake2 = [[Decimal(i) for i in j] for j in stake2] 

    return stake2

In [10]:
stake = setupStake()

In [11]:
stake2 = setupStake_e()

In [12]:
for p in range(numNodes):
        for q in range(numNodes):
            stake2[p][q] = stake[p][q]

In [13]:
stake

[[Decimal('0'),
  Decimal('0'),
  Decimal('31.300000000000000710542735760100185871124267578125'),
  Decimal('31.300000000000000710542735760100185871124267578125')],
 [Decimal('9.300000000000000710542735760100185871124267578125'),
  Decimal('0'),
  Decimal('9.300000000000000710542735760100185871124267578125'),
  Decimal('9.300000000000000710542735760100185871124267578125')],
 [Decimal('22.5'), Decimal('22.5'), Decimal('0'), Decimal('22.5')],
 [Decimal('3'), Decimal('3'), Decimal('3'), Decimal('0')]]

In [14]:
stake2

[[Decimal('0'),
  Decimal('0'),
  Decimal('31.300000000000000710542735760100185871124267578125'),
  Decimal('31.300000000000000710542735760100185871124267578125'),
  Decimal('0')],
 [Decimal('9.300000000000000710542735760100185871124267578125'),
  Decimal('0'),
  Decimal('9.300000000000000710542735760100185871124267578125'),
  Decimal('9.300000000000000710542735760100185871124267578125'),
  Decimal('0')],
 [Decimal('22.5'),
  Decimal('22.5'),
  Decimal('0'),
  Decimal('22.5'),
  Decimal('0')],
 [Decimal('3'), Decimal('3'), Decimal('3'), Decimal('0'), Decimal('0')],
 [Decimal('0'), Decimal('0'), Decimal('0'), Decimal('0'), Decimal('0')]]

In [15]:
calcImportance(stake)

array([Decimal('496.4677897157100818711496600'),
       Decimal('308.1128679068750630570542833'),
       Decimal('631.1010789538536399776299603'),
       Decimal('111.2491016713337106705887072')], dtype=object)

In [16]:
calcImportance(stake2)

array([Decimal('496.4677897157100818711496600'),
       Decimal('308.1128679068750630570542833'),
       Decimal('631.1010789538536399776299603'),
       Decimal('111.2491016713337106705887072'), Decimal('0')],
      dtype=object)

In [17]:
# 2 extra non-random node: numNodes-1 strategy:equally spreading between channels, numNodes-2: targeting the highest importance node and opening only one channel towards 

In [18]:
def calcImportance2(stake):

    # get total stake per node
    stakePerNode = [(sum(e)) for e in stake]

    totalStake = sum(stakePerNode)

    n = len(stake)
    weight = [0] * n
    importance = [0] * n
    for x in range(n):
        for y in range(n):
            # weight of a node is 0 if its stake is equal to 0
            #weight(channel) = sqrt(balance(channel) / stake(channel.source) * stake(channel.destination))
            weight[y] += 0 if stakePerNode[y]==0 else numpy.sqrt(stake[y][x]/stakePerNode[y] * stakePerNode[x])

    #print("Weighted downstream stake per node p(n) = ", weight)
    importanceList = numpy.array(weight) * numpy.array(stakePerNode)
    return importanceList;

def setupStake(
        # Number of nodes in the network. This number can be changed to test with small and large network size
        numNodes=7,
        # the minimum number of channels a node can open
        minChannelsPerNode=2,
        # the maximum number of channels a node can open 
        maxChannelsPerNode=10,
        minFundsPerNode=10,
        maxFundsPerNode=100,
        tokensPerTicket=0.1
    ):

    stake = [[0 for i in range(numNodes)] for j in range(numNodes)]
    listStakePerChannel = []
    avg_stake_list = []
    index_list = []
    imp_list = []
    for x in range(numNodes-2):
        # each node is given a random funding amount
        myFunds = numpy.random.rand() * (maxFundsPerNode - minFundsPerNode) + minFundsPerNode

        # get random number of channels per node
        myChannels = int(numpy.random.rand() * (maxChannelsPerNode - minChannelsPerNode + 1) + minChannelsPerNode)

        # This value represents the amount each node stakes in their channel
        # It is computed as the number of funds a node has divided by number of channels they open
        stakePerChannel = myFunds / myChannels
        stakePerChannel = int(stakePerChannel / tokensPerTicket) * tokensPerTicket
        
        # Creats a list of stakePerChannel
        listStakePerChannel.append(stakePerChannel)

        # Identifying the maximum stake
        maxListStakePerChannel = max(listStakePerChannel)
        
        # Identifying the "position" of the stake - which node initiated 
        index = listStakePerChannel.index(maxListStakePerChannel)
        index_list.append(index)
        index_max = max(index_list)

        # fund channels by writing into stake matrix
        for c in range(myChannels):
            # TODO: this does not prevent a node from opening a channel to the same counterparty multiple times
            counterparty = int(numpy.random.rand() * (numNodes - 3))

            # cannot open channel to self - keep diagonal of matrix at 0
            if counterparty >= x:
                counterparty = counterparty + 1
            stake[x][counterparty] = stakePerChannel
        
        #calculating the stake per channel
        sum_stake = sum([sum(stake[li]) for li in range(numNodes-2)])
        avg_stake = sum_stake/(numNodes-2)
        
        imp = calcImportance2(stake)
      
    #calculating imporance score
    imp_max = max(imp)
    ind = numpy.where(imp == imp_max)
    ma = ind[0][0]
  
    #non-random node 1
    stake[numNodes-2][ma] = avg_stake 
     
    #non-random nore 2
    for b in range(numNodes-2):
        stake[numNodes-1][b] = avg_stake/(numNodes-2)
    
    stake = [[Decimal(i) for i in j] for j in stake]
    

    return stake

In [19]:
z = setupStake()
z

[[Decimal('0'),
  Decimal('0'),
  Decimal('7.9000000000000003552713678800500929355621337890625'),
  Decimal('7.9000000000000003552713678800500929355621337890625'),
  Decimal('7.9000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('0')],
 [Decimal('3.600000000000000088817841970012523233890533447265625'),
  Decimal('0'),
  Decimal('0'),
  Decimal('3.600000000000000088817841970012523233890533447265625'),
  Decimal('0'),
  Decimal('0'),
  Decimal('0')],
 [Decimal('13.60000000000000142108547152020037174224853515625'),
  Decimal('0'),
  Decimal('0'),
  Decimal('13.60000000000000142108547152020037174224853515625'),
  Decimal('13.60000000000000142108547152020037174224853515625'),
  Decimal('0'),
  Decimal('0')],
 [Decimal('2.9000000000000003552713678800500929355621337890625'),
  Decimal('2.9000000000000003552713678800500929355621337890625'),
  Decimal('2.9000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('2.900000000000000355271367880050092

In [20]:
im = calcImportance(z)
im

array([Decimal('192.0574929629562063206267464'),
       Decimal('42.12505304186515624543422839'),
       Decimal('294.8439649260183887253769320'),
       Decimal('105.4537127208482070315043299'),
       Decimal('156.1042007379790864544846810'),
       Decimal('129.4105022013283291865017852'),
       Decimal('195.5950380285171525920528014')], dtype=object)

In [29]:
def calcImportance_pre(stake):

    # get total stake per node
    stakePerNode = [(sum(e)) for e in stake]

    totalStake = sum(stakePerNode)

    n = len(stake)
    weight = [0] * n
    importance = [0] * n
    for x in range(n):
        for y in range(n):
            # weight of a node is 0 if its stake is equal to 0
            #weight(channel) = sqrt(balance(channel) / stake(channel.source) * stake(channel.destination))
            weight[y] += 0 if stakePerNode[y]==0 else numpy.sqrt(stake[y][x]/stakePerNode[y] * stakePerNode[x])

    #print("Weighted downstream stake per node p(n) = ", weight)
    importanceList = numpy.array(weight) * numpy.array(stakePerNode)
    return importanceList;

def calcImportance(stake):

    # get total stake per node
    stakePerNode = [Decimal(sum(e)) for e in stake]

    totalStake = sum(stakePerNode)

    n = len(stake)
    weight = [0] * n
    importance = [0] * n
    for x in range(n):
        for y in range(n):
            # weight of a node is 0 if its stake is equal to 0
            #weight(channel) = sqrt(balance(channel) / stake(channel.source) * stake(channel.destination))
            weight[y] += 0 if stakePerNode[y]==0 else numpy.sqrt(stake[y][x]/stakePerNode[y] * stakePerNode[x])

    #print("Weighted downstream stake per node p(n) = ", weight)
    importanceList = numpy.array(weight) * numpy.array(stakePerNode)
    return importanceList;

def setupStake(
        # Number of nodes in the network. This number can be changed to test with small and large network size
        numNodes=7,
        # the minimum number of channels a node can open
        minChannelsPerNode=2,
        # the maximum number of channels a node can open 
        maxChannelsPerNode=10,
        minFundsPerNode=10,
        maxFundsPerNode=100,
        tokensPerTicket=0.1
    ):

    stake = [[0 for i in range(numNodes)] for j in range(numNodes)]
    listStakePerChannel = []
    avg_stake_list = []
    index_list = []
    imp_list = []
    for x in range(numNodes-3):
        # each node is given a random funding amount
        myFunds = numpy.random.rand() * (maxFundsPerNode - minFundsPerNode) + minFundsPerNode

        # get random number of channels per node
        myChannels = int(numpy.random.rand() * (maxChannelsPerNode - minChannelsPerNode + 1) + minChannelsPerNode)

        # This value represents the amount each node stakes in their channel
        # It is computed as the number of funds a node has divided by number of channels they open
        stakePerChannel = myFunds / myChannels
        stakePerChannel = int(stakePerChannel / tokensPerTicket) * tokensPerTicket
        
        # Creats a list of stakePerChannel
        listStakePerChannel.append(stakePerChannel)

        # Identifying the maximum stake
        maxListStakePerChannel = max(listStakePerChannel)
        
        # Identifying the "position" of the stake - which node initiated 
        index = listStakePerChannel.index(maxListStakePerChannel)
        index_list.append(index)
        index_max = max(index_list)

        # fund channels by writing into stake matrix
        for c in range(myChannels):
            # TODO: this does not prevent a node from opening a channel to the same counterparty multiple times
            counterparty = int(numpy.random.rand() * (numNodes - 4))

            # cannot open channel to self - keep diagonal of matrix at 0
            if counterparty >= x:
                counterparty = counterparty + 1
            stake[x][counterparty] = stakePerChannel
        
        sum_stake = sum([sum(stake[li]) for li in range(numNodes-3)])
        avg_stake = sum_stake/(numNodes-3)
        
        imp = calcImportance_pre(stake)
       
    #calculating imporance score
    imp_max = max(imp)
    ind = numpy.where(imp == imp_max)
    ma = ind[0][0]
  
    #non-random node 1
    stake[numNodes-3][ma] = avg_stake 
        
    #non_random node 2
    for b in range(numNodes-3):
        stake[numNodes-2][b] = avg_stake/(numNodes-3)
        
    #non_random node 3    
    for r in range(numNodes-3):
        stake[numNodes-1][r] = sum(stake[r])/sum_stake*avg_stake
    
    stake = [[Decimal(i) for i in j] for j in stake]
    
    return calcImportance(stake)

In [30]:
f = setupStake()
f

array([Decimal('716.1797977353527110517991577'),
       Decimal('572.6075885299838406111548381'),
       Decimal('290.4563169973306295944379846'),
       Decimal('418.3018581648136276013732291'),
       Decimal('416.9510516835280129430572231'),
       Decimal('633.5604870658492111303778283'),
       Decimal('647.5190371718811432041052875')], dtype=object)

In [31]:
def selectChannel(weights, weightIndexToNodeLUT):
    """
    Randomly selects a counterparty based on weights according to a LUT
    Returns the node id (or -1 if none could be found)
    Parameters
    ----------
    weights: a list of weights to be used for normalizing the random selection
    weightIndexToNodeLUT: a LUT translating the weights list to node ids
    """
    rand = numpy.random.rand()
    totalWeight = sum(weights)
    sumWeights = 0
    counterparty = -1
    for i in enumerate(weights):
        sumWeights += i[1]
        if totalWeight == 0:
           sumWeights = 0
        else:
            if sumWeights / totalWeight >= rand:
               counterparty = weightIndexToNodeLUT[i[0]]
               break;
    return counterparty

In [32]:
selectChannel(im, [i for i in range(len(im))])

4

In [33]:
def randomPickWeightedByImportance(importance):
    channel = selectChannel(importance, [i for i in range(len(importance))])
    return channel

In [34]:
randomPickWeightedByImportance(im)

6

In [35]:
 def sendPacket(stake, importance):
        print("sending packet")

        # persist importance list between attempts so that dead ends can be removed
        importanceAttempts = importance.copy()
        attemptsPerTick = 10
        hops = 3
        numPlayers = 4
        ctNodeId = 6

        for a in range(attemptsPerTick):
            nextNodeIndex = ctNodeId
            pathIndices = [nextNodeIndex]

            # try to find a path
            for j in range(hops):
                # reset importance
                importanceTmp = importanceAttempts.copy()
                #hoprsim.printArray1d(importanceTmp, 1)

                # remove importance entries for nodes to which current hop has no open channels
                # this is used in the path selection for the next hop
                for i in range(numPlayers):
                    if stake[nextNodeIndex][i] == 0 :
                        importanceTmp[i] = 0

                # prevent loops in path by removing existing nodes on path from list
                for i in pathIndices:
                    importanceTmp[i] = 0

                #hoprsim.printArray1d(importanceTmp, 1)
                nextNodeIndex = randomPickWeightedByImportance(importanceTmp)
                if nextNodeIndex == -1:
                    break # stop looking for path if no next node could be found
                pathIndices.append(nextNodeIndex)

            print("Found path: ", pathIndices)
            # then facilitate they payout to each node
            # but only as long as the edge is valid (new earnings + existing earnings <= counter party stake)

In [36]:
 sendPacket(z, im)

sending packet
Found path:  [6, 1, 3, 2]
Found path:  [6, 0, 2, 5]
Found path:  [6, 0, 4, 2]
Found path:  [6, 0, 4, 5]
Found path:  [6, 2, 0, 5]
Found path:  [6, 0, 2, 5]
Found path:  [6, 4, 0, 2]
Found path:  [6, 4, 0, 3]
Found path:  [6, 1, 5, 2]
Found path:  [6, 2, 5, 4]


In [37]:
def tick(stake, importance):
    ctTickDurationSeconds = 2
    print("CT tick")
    #self.openChannels()
    sendPacket(stake, importance)
    nextTick = datetime.datetime.utcnow() + datetime.timedelta(seconds = int(ctTickDurationSeconds))
    print("next tick: ", nextTick)
    t = threading.Timer(ctTickDurationSeconds, tick)
    t.start()

In [38]:
tick(z, im)

CT tick
sending packet
Found path:  [6, 0, 5, 2]
Found path:  [6, 5, 2, 0]
Found path:  [6, 5, 2, 3]
Found path:  [6, 2, 4, 5]
Found path:  [6, 0, 2, 5]
Found path:  [6, 2, 0, 4]
Found path:  [6, 4, 0, 2]
Found path:  [6, 3, 4, 0]
Found path:  [6, 0, 2, 3]
Found path:  [6, 3, 5, 2]
next tick:  2022-01-07 23:15:38.786006


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\szany\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\szany\Anaconda3\lib\threading.py", line 1158, in run
    self.function(*self.args, **self.kwargs)
TypeError: tick() missing 2 required positional arguments: 'stake' and 'importance'



In [39]:
def selectChannel(weights, weightIndexToNodeLUT):
    """
    Randomly selects a counterparty based on weights according to a LUT
    Returns the node id (or -1 if none could be found)
    Parameters
    ----------
    weights: a list of weights to be used for normalizing the random selection
    weightIndexToNodeLUT: a LUT translating the weights list to node ids
    """
    rand = numpy.random.rand()
    totalWeight = sum(weights)
    sumWeights = 0
    counterparty = -1
    for i in enumerate(weights):
        sumWeights += i[1]
        if totalWeight == 0:
           sumWeights = 0
        else:
            if sumWeights / totalWeight >= rand:
               counterparty = weightIndexToNodeLUT[i[0]]
               break;
    return counterparty;

def randomPickWeightedByImportance(importance):
    channel = selectChannel(importance, [i for i in range(len(importance))])
    return channel;

def openChannels(importance, stake):
        #print("opening channels")
        numOpenChannels = 0
        channelcount = 3
        ctn = 6
        openchannelid = [ctn]
        tokensPerChannel = 10
        # add own id to avoid self-staking
        while (numOpenChannels < channelcount):

            # do not open channels to same counterparty multiple times
            tmpImportance = list(importance)
            for i in range(len(importance)):
                if i in openchannelid:
                    tmpImportance[i] = 0

            newChannel = randomPickWeightedByImportance(tmpImportance)
            
            if (newChannel == -1):
                print("ERROR: could not find a counterparty to open the channel to!")
                # TODO: here we should try again but this time including 0 importance nodes
            else:
                print("opening channel to node ", newChannel)
                stake[ctn][newChannel] = tokensPerChannel
                openchannelid.append(newChannel)
            # if opening channels didn't work we still want this loop to terminate
            numOpenChannels = numOpenChannels + 1
        
        return openchannelid

In [40]:
openChannels(im, z)

opening channel to node  0
opening channel to node  3
opening channel to node  2


[6, 0, 3, 2]

In [43]:
results = []
for _ in range(1000):
    opening_channel_to = openChannels(im, z)
    results.append(opening_channel_to)

opening channel to node  0
opening channel to node  5
opening channel to node  1
opening channel to node  4
opening channel to node  2
opening channel to node  5
opening channel to node  3
opening channel to node  0
opening channel to node  5
opening channel to node  2
opening channel to node  0
opening channel to node  5
opening channel to node  2
opening channel to node  0
opening channel to node  4
opening channel to node  2
opening channel to node  5
opening channel to node  0
opening channel to node  2
opening channel to node  0
opening channel to node  5
opening channel to node  2
opening channel to node  5
opening channel to node  3
opening channel to node  0
opening channel to node  4
opening channel to node  5
opening channel to node  2
opening channel to node  4
opening channel to node  0
opening channel to node  2
opening channel to node  1
opening channel to node  3
opening channel to node  5
opening channel to node  4
opening channel to node  2
opening channel to node  2
o

opening channel to node  2
opening channel to node  0
opening channel to node  4
opening channel to node  5
opening channel to node  5
opening channel to node  2
opening channel to node  0
opening channel to node  2
opening channel to node  3
opening channel to node  5
opening channel to node  2
opening channel to node  3
opening channel to node  0
opening channel to node  0
opening channel to node  5
opening channel to node  4
opening channel to node  5
opening channel to node  2
opening channel to node  4
opening channel to node  4
opening channel to node  5
opening channel to node  2
opening channel to node  2
opening channel to node  0
opening channel to node  4
opening channel to node  3
opening channel to node  0
opening channel to node  2
opening channel to node  2
opening channel to node  3
opening channel to node  4
opening channel to node  1
opening channel to node  2
opening channel to node  5
opening channel to node  0
opening channel to node  2
opening channel to node  4
o

opening channel to node  3
opening channel to node  2
opening channel to node  2
opening channel to node  0
opening channel to node  4
opening channel to node  4
opening channel to node  1
opening channel to node  5
opening channel to node  3
opening channel to node  0
opening channel to node  2
opening channel to node  0
opening channel to node  4
opening channel to node  3
opening channel to node  2
opening channel to node  0
opening channel to node  1
opening channel to node  2
opening channel to node  4
opening channel to node  3
opening channel to node  2
opening channel to node  4
opening channel to node  5
opening channel to node  2
opening channel to node  5
opening channel to node  0
opening channel to node  0
opening channel to node  4
opening channel to node  5
opening channel to node  3
opening channel to node  0
opening channel to node  1
opening channel to node  4
opening channel to node  2
opening channel to node  1
opening channel to node  0
opening channel to node  5
o

opening channel to node  3
opening channel to node  4
opening channel to node  0
opening channel to node  0
opening channel to node  5
opening channel to node  4
opening channel to node  5
opening channel to node  2
opening channel to node  4
opening channel to node  2
opening channel to node  0
opening channel to node  5
opening channel to node  0
opening channel to node  2
opening channel to node  3
opening channel to node  3
opening channel to node  2
opening channel to node  5
opening channel to node  3
opening channel to node  0
opening channel to node  5
opening channel to node  0
opening channel to node  3
opening channel to node  2
opening channel to node  2
opening channel to node  0
opening channel to node  3
opening channel to node  3
opening channel to node  0
opening channel to node  5
opening channel to node  4
opening channel to node  2
opening channel to node  1
opening channel to node  4
opening channel to node  5
opening channel to node  2
opening channel to node  2
o

In [44]:
results_pd = pd.DataFrame(results)

In [45]:
b = []
for h in range(len(results_pd.columns)):
    a = results_pd[h].value_counts()
    b.append(a)
b = pd.concat(b).to_frame().reset_index()
b.columns = ['opening', 'sum']

In [46]:
b.groupby(['opening']).sum()

,sum
opening,
0,636
1,195
2,758
3,395
4,502
5,514
6,1000


In [47]:
im

array([Decimal('192.0574929629562063206267464'),
       Decimal('42.12505304186515624543422839'),
       Decimal('294.8439649260183887253769320'),
       Decimal('105.4537127208482070315043299'),
       Decimal('156.1042007379790864544846810'),
       Decimal('129.4105022013283291865017852'),
       Decimal('195.5950380285171525920528014')], dtype=object)